In [1]:
# Preprocess and download datasets

# Variables
DATASET_PATH = './data/'
RESULT_PATH = './result_full2/'

EPOCHS = 15

# Import the necessary libraries
import numpy as np
import torch
import matplotlib.pyplot as plt
import json
import time
import tqdm

# Loading the MNIST dataset
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize((224, 224)),
                                transforms.Normalize((0.5), (0.5))
                               ])

# Download and load the training data
trainset = datasets.MNIST(DATASET_PATH, download = True, train = True, transform = transform)
testset = datasets.MNIST(DATASET_PATH, download = True, train = False, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 16, shuffle = True)
testloader = torch.utils.data.DataLoader(testset, batch_size = 16, shuffle = True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# Training and evaluating process

# Initiate the timer to instrument the performance
timer_start = time.process_time_ns()
epoch_times = [timer_start]

train_losses, test_losses, accuracies = [], [], []

for e in range(EPOCHS):
    running_loss = 0
    print("Epoch: {:03d}/{:03d}..".format(e+1, EPOCHS))
    # Training pass
    print("Training pass:")
    for data in tqdm(trainloader, total=len(trainloader)):
        images, labels = data
        
        output = model.forward(images)
        loss = criterion(output, labels).to(device)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    # Testing pass
    print("Validation pass:")
    test_loss = 0
    accuracy = 0
    # Turn off gradients for validation, saves memory and computation
    with torch.no_grad():
        # Set the model to evaluation mode
        model.eval()
        
        # Validation pass
        for data in tqdm(testloader, total=len(testloader)):
          images, labels = data
          images = images.to(device)
          labels = labels.to(device)
          labels_one_hot = to_one_hot(labels, device)
          log_ps = model(images).to(device)
          test_loss += criterion(log_ps, labels_one_hot)
          
          ps = torch.exp(log_ps).to(device)
          top_p, top_class = ps.topk(1, dim = 1)
          equals = (top_class == labels.view(*top_class.shape))
          accuracy += torch.mean(equals.type(torch.FloatTensor))
  
    model.train()
    train_losses.append(running_loss/len(trainloader))
    test_losses.append(float(test_loss.cpu())/len(testloader))
    accuracies.append(float(accuracy)/len(testloader))
    
    epoch_times.append(time.process_time_ns())
    print("Training loss: {:.3f}..".format(running_loss/len(trainloader)),
          "Test loss: {:.3f}..".format(test_loss/len(testloader)),
          "Test Accuracy: {:.3f}".format(accuracy/len(testloader)),
          "Cur time(ns): {}".format(epoch_times[-1]))

fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(train_losses, label="Training loss")
ax.plot(test_losses, label="Validation loss")
ax.set_xlabel("Epoch")
ax.set_ylabel("Cross Entropy Loss")
ax.legend()
ax2 = ax.twinx()
ax2.plot(np.array(accuracies)*100, label="Accuracy", color='g')
ax2.set_ylabel("Accuracy (%)")
plt.title(plt_title)
plt.savefig(RESULT_PATH + 'training_proc.png', dpi = 100)

with open(RESULT_PATH + 'torch_results.json', 'w+') as f:
    json.dump({
        'train_losses': train_losses,
        'test_losses': test_losses,
        'epoch_times': epoch_times,
        'accuracies': accuracies,
      }, f)